In [1]:
import numpy as np
import graphinglib as gl
from collections import namedtuple
import astropy.units as u

from src.hdu.cubes.cube_co import CubeCO
from src.spectrums.spectrum_co import SpectrumCO
from src.spectrums.spectrum import Spectrum
from src.hdu.tesseract import Tesseract
from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.convenient_funcs import get_FWHM, get_speed
from src.coordinates.ds9_coords import DS9Coords

# Loop4N1

In [ ]:
N1 = CubeCO.load("data/Loop4_co/N1/Loop4N1_FinalJS.fits")[500:800,:,:].bin((1,2,2))
N1.header["COMMENT"] = "Loop4N1_FinalJS was binned 2x2."
N1.header["COMMENT"] = "Loop4N1_FinalJS was sliced at channel 500; all values of mean must then be " \
                        + "added to 500 to account for this shift."
# N1.save("data/Loop4_co/N1/Loop4N1_FinalJS_bin2.fits")

## Fitting

In [ ]:
if __name__ == "__main__":
    spectrum_parameters = {
        "PEAK_PROMINENCE" : 0.3,
        "PEAK_MINIMUM_DISTANCE" : 6,
        "PEAK_WIDTH" : 2,
        "INITIAL_GUESSES_BINNING" : 2,
        "MAX_RESIDUE_SIGMAS" : 6,
        "INITIAL_GUESSES_MAXIMUM_GAUSSIAN_STDDEV" : 10
    }

    chi2, fit_results = N1.fit(spectrum_parameters)
    chi2.save("data/Loop4_co/N1/chi2.fits")
    fit_results.save("data/Loop4_co/N1/tesseract.fits")

## Slicing

In [ ]:
print(f"Targeted channel : {N1.header.get_coordinate(-4000, 0)}")
fit_results = Tesseract.load("data/Loop4_co/N1/tesseract.fits")

splits = ["lower_left", "lower_right", "upper"]
tesseract_splits = namedtuple("tesseract_splits", splits)
lower, upper = fit_results.split(14, 2)
lower_left, lower_right = lower.split(10, 3)
tesseract_splits = tesseract_splits(lower_left, lower_right, upper)

for split in splits:
    getattr(tesseract_splits, split).save(f"data/Loop4_co/N1/tesseract_splits/{split}.fits")

upper = tesseract_splits.upper.filter(slice(200, None))
lower_left = tesseract_splits.lower_left.filter(slice(197, None))
lower_right = tesseract_splits.lower_right.filter(slice(190, None))

lower = lower_left.concatenate(lower_right, 3)
total_object_N1 = lower.concatenate(upper, 2)

# Compressing the Tesseract
total_object_N1 = total_object_N1.compress()
total_object_N1.save(f"data/Loop4_co/N1/object.fits")

## Verifications

In [ ]:
total_object_N1 = Tesseract.load(f"data/Loop4_co/N1/object.fits")
fig = gl.Figure(size=(10,7), figure_style="dim")
fig.add_elements(*total_object_N1.get_spectrum_plot(N1, DS9Coords(11, 10)))
fig.show()

## Refits

In [ ]:
# Working in DS9Coods
spectrums = {
    (11, 10) : N1[:,*DS9Coords(11, 10)],
    (14, 10) : N1[:,*DS9Coords(14, 10)]
}

for coords, spectrum in spectrums.items():
    total_object_N1[0,*DS9Coords(*coords)] = np.NAN

total_object_N1.compress().save("data/Loop4_co/N1/object_filtered.fits")